# Explanation of my solution

In this notebook, I am going through the solution code in its order of execution as well as through the thinking process I had to find this solution to the problem.

I will name this solution: **Moving the chairs to the room names**.

You can follow this code along in the `script.py` file at the root of the directory.



In [1]:
from utils_notebook import get_project_root, notebook_print_of

ROOT = get_project_root()
EXAMPLES_FOLDER = str(ROOT) + '/explanatory_rooms'

## Overall organization of the code

In the `script.py` file we have as:
- **Input**: a path to the `txt` file reprensenting an apartment. *ex: rooms.txt*
- **Output**: a print in the console of the excepted result, namely the:
```
- Number of different chair types for the apartment
- Number of different chair types per room
```
as mentioned in the **task_en.txt** file.

To perform this task, it runs the `run_solution()` function

The `run_solution()` is divided in 4 steps:

- **Step 1:** Read the `txt` file
- **Step 2:** Locate the chairs in the apartment
- **Step 3:** Find the room of each chair
- **Step 4:** Transform the result and display it

## Preliminary steps

### Step 1: Read and transform the txt file

I simply read the `room.txt` file or any other `txt` file of the same type and tranform it into a list of lists that I name `apartment_init`. 

I chose this *list of lists* data structure, i.e. to easily track any point in the apartment with coordinates: i, j as follows: `apartment_init[i][j]` which will be used in the rest of the code.

For debugging purposes, note that an apartment as a list of lists can be displayed in the console or the debugger with the function `console_print_of()` present in the `utils.py` file.

### Step 2: Locate chairs in the apartment

1. I scan throught every single pair of coordinates `(i, j)` to build the dictionary: `dict_pos_chairs` with:
    - **key**: (i, j) the coordinates of the chairs
    - **values**: the coresponding letter of chair in the apartment

2. In addition to this dictionary, I also create a list `list_pos_chairs` which gives the positions of chairs ordered vertically and horizontally. Sorting the list this way allows to easily find which chair is treated when we loop over each of them.



## Step 3: Find the room of each chair 
### (Main step)

This step is divided into 3 sub-steps:
- **Step 3.1:** Temporarily remove the other chairs in the apartment
- **Step 3.2:** Search the room of a chair
- **Step 3.3:** Save the result

This hole step is performed looping over the position of the chairs: `list_pos_chairs`. Within the loop the variables are:
- `k`: the index in `list_pos_chairs` of the chair. Having the `txt` file open, as `list_pos_chairs` is sorted it is fairly easy to locate the chair with `k`.
- `coord = (i, j)` : the coordinates of the chair in the `apartment`
-  `chair = apartment_init[i][j]`: the letter of the chair


### Step 3.1: Temporarily remove the other chairs in the apartment

We are now inside the **for** loop and therefore the variables above-mentioned will be used in the explanation.

As explained in the introduction, my method consists in moving the letter of each chair. To avoid any kind of conflicts between two chairs letters, I decided to create a copy (or rather a [deepcopy](https://docs.python.org/3/library/copy.html)) of `apartment_init` called `apartment` in which the other chair letters than `chair` will be removed.

This is how `apartment` looks like when treating the chair `'S'` located in `i, j = (4, 10)`

In [18]:
notebook_print_of(f'{EXAMPLES_FOLDER}/1.rooms_one_chair.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '| (closet)  |                                    |',
 '|           |                            S       |',
 '|           |         (sleeping room)            |',
 '|           |                                    |',
 '|           |                                    |',
 '+-----------+                                    |',
 '|           |                                    |',
 '|           |                                    |',
 '|           |                                    |',
 '|           +--------------+---------------------+',
 '|                          |                     |',
 '|                          |                     |',
 '|                          |    (office)         |',
 '|                          |                     |',
 '+--------------+           |                     |',
 '|              |           |                     |',
 '| (toile

### Step 3.2: Search the room of a chair

#### (The core of the algorithm)

For this step, we call the function `search_room()` that has as inputs:
- `apartment`: the apartment with only on chair present
- `coord`: the coordinates of the chair
an return:
- `room_of_chair`: the room name, the chair belongs to

In the above example:
```
search_room(apartment, (4, 10)) = 'sleeping room'
```

The best way to explain the logic behind this piece of code is I think to go through the different ideas that I had.

#### IDEA 1: room name on the same row

The idea consists of:

1. Checking if a chair located in `(i, j)` is in the same row as its room name.
2. Saving this room name

Building the object `apartment` as a list of lists, it fairly easy to extract a given row `i`. We just need to define: `row = apartment[i]`

If we consider this situation:

In [19]:
notebook_print_of(f'{EXAMPLES_FOLDER}/2.chair_room_same_row.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '|           |                                    |',
 '|  (closet) |                                    |',
 '|           |                                    |',
 '|           |                                    |',
 '|           |                                    |',
 '+-----------+                                    |',
 '            |         (sleeping room)     S      |',
 '            |                                    |',
 '            |                                    |',
 '            +------------------------------------+']

In this situation, asserting if the chair has its room name on the same row can be done by checking if a paranthesis belongs to this row.

However, if we are in this situation:

In [20]:
notebook_print_of(f'{EXAMPLES_FOLDER}/3.not_good_room_same_row.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '|           |                                    |',
 '|           |                                    |',
 '| (closet)  |              S                     |',
 '|           |                                    |',
 '|           |                                    |',
 '+-----------+                                    |',
 '            |              (sleeping room)       |',
 '            |                                    |',
 '            |                                    |',
 '            +------------------------------------+']

Visually, it is obvious to say that the chair `S` is in the room `sleeping room`. But the row where `S` is contains another room name: `closet`.

The strategy of finding a paranthesis on the same row, will give a false result in this case.

The right approach is:
- to check if a room name is on the same row as a chair
- if so, check if exclusively `' '` characters are between the chair letter and the room name on the same row.

This is performed by the function `is_room_on_same_row()`, with an equivalent method:
- we remove `' '` from the row
- we check if directly preceded by a `)` or followed by a `(` as room names are wrapped in them.

As the second step of this first idea, if `is_room_on_same_row(apartment, i, j)` has a `True` value, we want to save the found room name. 

This task is performed by the function `found_room_on_same_row()` in this way, we go through the `row = apartment[i]`:

1. We start from the chair in `row[j]`
2. We create two indices `j_left` and `j_right`
3. We move `j_left` on the left and `j_right` on the right, while no paranthesis is found. It obviously terminates as `is_room_on_same_row(apartment, i, j)` is `True`. 

***Note:*** *Notice that the value of `j_left` and `j_right` are only updated when not encountering an element of `sep_chars`. This ensures to only stay within the room*. `sep_chars` are the characters of separation of a room that you can find in the `special_characters.py` file.

4. Once a paranthesis is encountered, we set the string `side` which is in `{'left', 'right'}` depending on where the paranthesis was found.
5. We finally build the room name character after character on either side depending on the value of `side`.

#### IDEA 2: Moving vertically and explore horizontally

So far, I have been able to find a room name of a chair if they were both on the same line. But what if that was not the case as in the example below?

In [21]:
notebook_print_of(f'{EXAMPLES_FOLDER}/4.room_not_on_same_row.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '|           |                                    |',
 '|           |                                    |',
 '| (closet)  |                                    |',
 '|           |                                    |',
 '|           |      S                             |',
 '+-----------+                                    |',
 '            |              (sleeping room)       |',
 '            |                                    |',
 '            |                                    |',
 '            +------------------------------------+']

The idea I got, was to vertically displace the chair so that it lands on a row where its room name is.

This deplacement is performed by the function `change_pos()` which has as inputs:
- `apartment`
- `i`, `j`: the old coordinates
- `new_i`, `new_j`: the coordinates of the spot we want to move the chair to

The function:
1. switches the characters at `(i, j)` and `(new_i, new_j)`
2. returns the updated coordinates.

Now that we know how to move a chair around, let's see how we can move vertically to find the room name.

I decided to:

1. Move the chair to the empty space just above it until we reach the top of the room, i.e., the above character is in `sep_chars`.

2. If the top of the room is reached, get back to the initial position.

3. Move the chair to the empty space just below it until we reach the bottom of the room, i.e., the below character is in `sep_chars`.

At each step of this procedure, we check if the room name is on the same row, we break it.


This task is perfomed the `explore_vertical_moves()` function. This function does other things on the side that we will explain later but the core of the function is explained above.

Here are a few examples. The numbers give the positions in order of the different positions that the chair `S` takes. 


In [22]:
notebook_print_of(f'{EXAMPLES_FOLDER}/5.horizontal_exploration_1.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '| (closet)  |    (sleeping room)             3   |',
 '|           |                                2   |',
 '+-----------+                                1   |',
 '            |                                S   |',
 '            |                                    |',
 '            +------------------------------------+']

In [23]:
notebook_print_of(f'{EXAMPLES_FOLDER}/6.horizontal_exploration_2.txt')

['+-----------+------------------------------------+',
 '|           |      2                             |',
 '| (closet)  |      1                             |',
 '|           |      S                             |',
 '+-----------+      3                             |',
 '            |      4       (sleeping room)       |',
 '            |                                    |',
 '            +------------------------------------+']

#### ALERT: Problem if the chair is above or under the room

Moving vertically and exploring horizontally seems to work fine, if of course we can move vertically. What if one of the character of the room name or one of the paranthesis that wraps it is on the same **column** as `chair`. If we try to move the chair up and down, it will eventually "bump into" the room name which will create a mess.

Here is an example of such a case:

In [24]:
notebook_print_of(f'{EXAMPLES_FOLDER}/7.chair_and_room_on_same column.txt')


['+-----------+------------------------------------+',
 '|           |                                    |',
 '| (closet)  |                 S                  |',
 '|           |                                    |',
 '+-----------+                                    |',
 '            |              (sleeping room)       |',
 '            |                                    |',
 '            +------------------------------------+']

The solution I found to this problem is to introduce one **vertical check** before moving the chair up and down in the `explore_vertical_moves()` funciton. This is performed by the `is_room_on_same_column()`

The approach is similar to that of `is_room_on_same_column()` but once the `' '` are removed, instead of looking for a paranthesis, we look for a character that is not in `sep_char`, i.e., a letter or a paranthesis.

In the example above, the column of `S` when the `' '` are removed, is:
`['-', 'S', 'e', '-']` as `'e'` follows `'S'` and is not in `sep_chars`, there is a room name on the same column.

***Note:*** *There is a side case to this problem of this problem. I will tackle it in the next notebook*

When the `is_room_on_same_column()` is `True`, there is no need to move the chair up and down. Instead we use the function `find_room_on_same_column()`.

This function, as its equivalent `find_room_on_same_row()`:

1. We start from the chair in `col[i]`
2. We create two indices `i_up` and `j_down`
3. We move `i_up` up and `i_dowm` down, until one of them reach a character of `no_room_char`, i.e. a paranthesis or a letter

***Note:*** *Notice that the value of `i_up` and `i_down` are only updated when not encountering an element of `sep_chars`. This ensures to only stay within the room*.


Once a character of a character in `no_room_char` is encountered:
4. We name the index where we found it `i_room`. Thus, the character is in `apartment[i_room][j]`

5. If it is a left paranthesis we change j = j+1
6. If it is a right paranthesis we change j = j-1 

So now `apartment[i_room][j]` is right in the middle of the room name.

7. From this character, we will set two variables `j_left` and `right` that we will move respectively left and right untile reaching the parantheses, so that we can build the room name character by character.

We finally return this room name.

#### IDEA 3: Find an open corner and vertical exploration

At this point, given the input `rooms.txt` file, I could find the room of all the chairs, except the `W` in the `living room` which is on the top far from its room name.

For this room, running `explore_horizontal_moves()`, `W` will move this way, without find the room name as in none of the rows `W` will be at, the room name will be present.

In [25]:
notebook_print_of(f'{EXAMPLES_FOLDER}/8.far_room_chair_moves.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '| (closet)  |                                    |',
 '|           |                                    |',
 '|           |         (sleeping room)            |',
 '|           |                                    |',
 '|           |                                    |',
 '+-----------+                                    |',
 '|        1  |                                    |',
 '|        W  |                                    |',
 '|        2  |                                    |',
 '|        3  +--------------+---------------------+',
 '|        4                 |                     |',
 '|        5                 |                     |',
 '|        6                 |    (office)         |',
 '|        7                 |                     |',
 '+--------------+           |                     |',
 '|              |           |                     |',
 '| (toile

It seems that `W` is confined to this region of the apartment. 

The solution I found for this problem is looking for an **open corner**.

What is an **open corner**?

An **open corner** is `+` on the same row as a given chair that forms a perpendicular angle, that expands the area of the room that you can explore.

Let's give an example:

In [26]:
notebook_print_of(f'{EXAMPLES_FOLDER}/9.open_corner_example.txt')

['+------------------------+-----------------------+',
 '|                        |                       |',
 '|            P2          |                       |',
 '|      P1   +-------+....|                       |',
 '|           | ~~~~~ |....|                       |',
 '|           | ~~~~~ |....|                       |',
 '|           | ~~~~~ +----+   S1                  |',
 '+-----------+          S2                        |',
 '            |                                    |',
 '            |                                    |',
 '            |                                    |',
 '            +------------------------------------+']

If we consider the chair `P1`. Running `explore_horizontal_moves()`, we will be unable to explore the area of its room "colored" with `'.'`. However if we change the position of `P1` to that of `P2`, we will get closer to the unexplored area. In this case the `+` on the right of `P1` is an open corner for `P1`.

Similarily for `S1` and the area "colored" with `'~'`. We will need to go to `S2` to get closer to the '~'-colored area. The  `+` on the left of `S1` is an open corner for `S1`.

The solution to get out of an area where the exploration is unsucessful is to move to a location in the direction that the **open corner** gives

Examples:
- The open corner of `P1` is open **up-right**. Let's move to `P2` which is **up-right**
- The open corner of `S1` is open **down-left**. Let's move to `S2` which is **down-left**

Well, we get closer to the '.'-colored are in `P2`, good! Yet,  running `explore_horizontal_moves()` from `P2` will not explore it. 

That is where we introduce the `explore_vertical_moves()` function. This function is conceptually similar to `explore_horizontal_moves()` in every way. You just need to switch "rows" by "columns" and vice versa.

In other words, from a `chair` in the apartment in `apartment[i][j]`:

1. We check, one time, if the room name is not on the same row as the chair with the function `is_room_on_same_row()`.
    - If so we find it witht the function `find_room_on_same_row()`
    
2. We move left and then right until reaching a character in `sep_chars`. At each iteration, we check `is_room_on_same_column()`
    - if it is `True`, we break with `find_room_on_same_column()`

To summarize, with the above example:

1. From `P1`, finding the open corner on the right
2. Moving `P1` to `P2`
3. From `P2` running `explore_vertical_moves()` 

will allow to find explore the are colored with `'.'`

#### IDEA 4: Open Corener generalisation

We know that in the cases where we are confined to a certain area, the open corners will allow us to to move to other areas. We may wonder how and when in the code we look for these open corners. 

**When?**

At each new itearation when the chair is moved in the while loops within the functions `explore_vertical_moves()` and `explore_horizontal_moves()`.

**How?**

With the functions `find_open_corner_horizontal_check()` and `find_open_corner_horizontal_check()`.


We will explain how the function `find_open_corner_horizontal_check()` called in the function `explore_vertical_moves()` works. The function `find_open_corner_vertical_check()` called in the function `explore_horizontal_moves()` works. follows the same exact process.

During a **vertical exploration**, 
For any location of chair you can find 0, 1 or 2 open corners. 
You'll find below an example for each of these cases.

The function `find_open_corner_horizontal_check()` has as inputs:
- `apartment`
- `(i, j)`: the coordinates of the chair
- `direction`: the direction the chair is currently move. In explore_vertical_moves(), it can be `up` or `down` 

It will return, the list of the coordinates of the found open corners on the same row as the chair, on its direct left and/or right.

**No Open Corners**

In the example below:
- At `S`
- During a **vertical exploration**
- With `direction = up`

There is no open corner



In [34]:
notebook_print_of(f'{EXAMPLES_FOLDER}/10.no_open_corner.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '| (closet)  |                 S                  |',
 '|           |                                    |',
 '+-----------+                                    |',
 '            |                                    |',
 '            |                                    +--------+',
 '            |                                             |',
 '    +-------+                                             |',
 '    |                                                     |',
 '    |                                                     |',
 '    +-------+                                    +--------+',
 '            |                                    |',
 '            |                                    |',
 '       +----+                                    |',
 '       |                                         |',
 '       +----+                                    |',
 '         

**One Open Corner**

In the example below:
- At `S`
- During a **vertical exploration**
- With `direction = down`

There is one open corner on the left.

However
- At `S`
- During a **vertical exploration**
- With `direction = up`

There is no open corner.

In [35]:
notebook_print_of(f'{EXAMPLES_FOLDER}/11.one_open_corner.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '| (closet)  |                                    |',
 '|           |                                    |',
 '+-----------+                                    |',
 '            |                                    |',
 '            |                                    +--------+',
 '            |                                             |',
 '    +-------+                                             |',
 '    |                                                     |',
 '    |                                                     |',
 '    +-------+                                    +--------+',
 '            |                                    |',
 '            |                                    |',
 '       +----+    S                               |',
 '       |                                         |',
 '       +----+                                    |',
 '         

During a **vertical exploration**, with the direction is `down`, there is no open corner.

**Tow Open Corners**

In the example below:
- At `S`
- During a **vertical exploration**
- With `direction = up`

There two open corners on the left and on the right.

In [36]:
notebook_print_of(f'{EXAMPLES_FOLDER}/12.two_open_corners.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '| (closet)  |                                    |',
 '|           |                                    |',
 '+-----------+                                    |',
 '            |                                    |',
 '            |                                    +--------+',
 '            |                                             |',
 '    +-------+                                             |',
 '    |                                                     |',
 '    |                                                     |',
 '    +-------+                S                   +--------+',
 '            |                                    |',
 '            |                                    |',
 '       +----+                                    |',
 '       |                                         |',
 '       +----+                                    |',
 '         

#### IDEA 5: Stacking the checkpoints

In this part, I will try to generalize the previous intuitive idea so that it can allow to find a room name from a chair position.

In the following, we will use the **checkpoint**. Let's define it first.

A **Checkpoint** is a dictionary that contains 2 keys.
- `coord`: the coordinates we want to move the chair to when we start an exploration.
- `exp_type`: the type of exporation that we want to perform: **vertical** or **horizontal**.

Then from the `search_room()` already we define lists of checkpoints that will be updated all along the research:
- `stack_checkpoints`: a list of the checkpoints that we want to explore. At a given time when we run the function `search_room()`, the future checkpoint that needs to be explored will be the last element of `stack_checkpoints` that waits to be poped out from the list. This element will also be the latest one added to the list. It is then a stack.
- `found_checkpoints`: a list of all the checkpoints that we have encountered so far. 

***Note:*** *It would have made more sense to create this object as a `set`. But `dict` objects being mutable and `set` objects, it is impossible to create a set of dictionaries in Python.*


Within the `search_room()` function, we initiate `stack_checkpoints` and `found_checkpoints` with two checkpoints:

1. The initial coordinates of the chair with a horizontal type of exploration
2. The initial coordinates of the chair with a vertical type of exploration

This ensures to either find the room directly in the area that we can explore or find at least one open corner during one of the explorations.

In the **while** loop of the same function, until we find the room:

1. We pop the last checkpoint in `stack_checkpoints` extract:
    - `(i, j)`: the coordinates of the checkpoint
    - exp_type: the type of exploration we want to perform.
    
2. We run the `explore()` with the defined variables: `apartment`, `exp_type`, `i`, `j`, `stack_checkpoints`, `found_checkpoints`. This `explore()` function will either run the `explore_horizontal_moves()` function or the `explore_verticalal_moves()` function depending on the value of `exp_type`. The `explore()` function will return `room_of_chair, (i, j)`:
    - The room name if found and if not the string `'not_found'`  
    
    - The coordinates of the chair at the end of the performed exploration. We keep these coordinates in case the room name was not found during the exploration so that we can run the `change_pos()` function at the next iteration of the loop with them and the coordinates of the next checkpoint.


The `stack_checkpoints` and `found_checkpoints` variables are  passed through the `explore()` function and then either to the `explore_vertical_moves()` function or `explore_vertical_moves()` function to be updated.

In these functions, wihin the while loop, after looking for **open corners**, we run the function `update_checkpoints()`.

This function takes as inputs:
- `stack_checkpoints` 
- `found_checkpoints`
- `new_open_corners`: the list of the open corners we found just before entering the function
- `direction_when_found`: the direction the chair was moving when it found the open corner in `{'right', 'left', 'up', 'down'}

We loop throuh the 0, 1 or 2 **open corners**.
For each of them we build the 2 `new_checkpoints` with:
- `coord`: "just around the open corner" (see the examples below)
- `exp_type`: one `'h'`, one `'v'`. We add the checkpoint with an `exp_type` different than the current one in the last place (ex: `direction_when_found = `left`, so the current exp_type is 'h', so we stack 'h' first and 'v' second for the next checkpoint). This is done to optimize the research.

Finally We loop over the the built checkpoints and for each checkpoint, we add it to `found_checkpoints` and `stack_checkpoints` only if it is not present in `found_checkpoints`. This avoids entering into the same procedures and not getting out the operattions.

In the example below you can find an example of a **vertical exploration**, from the chair `S`. The found checkpoints that will be stacked are names `c_n`, (c1 being found before c2 ...)

In [37]:
notebook_print_of(f'{EXAMPLES_FOLDER}/13.checkpoints_vertical_exploration.txt')

['+-----------+------------------------------------+',
 '|           |                 11                 |',
 '| (closet)  |                 10                 |',
 '|           |                  9                 |',
 '+-----------+                  8                 |',
 '            |                  7                 |',
 '            |                  6                 +--------+',
 '            |                  5                          |',
 '    +-------+                  4                          |',
 '    |                          3                          |',
 '    |      c1                  2                  c2      |',
 '    +-------+                  1                 +--------+',
 '            |                  S                 |',
 '            |                 12                 |',
 '       +----+                 13                 |',
 '       |   c3                 14                 |',
 '       +----+                 15                 |',
 '         

To have more insights on the moves of a chair, in more complex cases, I advise you to follow the instructions in the `README.md` file.

### Step 3.3: Save the result

The algorithmic strategy detailed above will allow to find the corresponding room name for each `coord` of chairs in the `apartment`. 

If we get back to the **For** loop of the **step 3**, this is performed with the `search_room()` function. The room name is then saved as the variable `room_of_chair`.

Right before we enter the **For** loop of the **step 3**, we initiate a `defauldict(list)` called `dict_rooms_chairs`. This dictionary will have as:
- **keys**: the room names
- **values**: the list of the chairs in the room chairs

It will be updated in the loop after finding the `room_of_chair` by appending `chair` to the value of the key `room_of_chair`, which is a list. We use the object `defauldict` so that we don't have to initialize the keys with empty lists at any time.


## Step 4: Transform the result and display it

The object `dict_rooms_chairs` contains all the information needed to generate the output in the terminal. Now, we just need to focus on how to transform it so that it meets the requiremnts of the `task_en.py` file.

1. From `dict_rooms_chairs`, we make the dictionary `grouped_dict_room_chairs` with the function `group_dict_room_chairs` which has as:
- **keys:** the rooms of the apartment
- **values**: the dictionary which gives for each chair its count in the room.

***Note:*** *The room that are not present in a given room will be given a count of 0*

2. From `dict_rooms_chairs`, we make the dictionary `dict_total_chairs` with the function `make_total_dict_result` which has as:
- **keys:** the chairs in `apartment_init`
- **values**: their count in `apartment_init`

3. As we could from the first notebook already make this dictionary of total results without running this algorithm, we can now compare the results before and after it. This is performed by the fucntion `check_total_chairs`.
 Having a match between these two dictionaries is a necessary condition for getting the correction result, that's why, we run an assertion.

 4. When the match is checked, we create the dictionary: `grouped_dict_final_output` which is `grouped_dict_rooms_chairs to which we add the key-value pair:
 - **key:** `'total'`
 - **value:** `dict_total_chairs`

`grouped_dict_final_output` contains the information of every pair of rows of the final output in the console, as key-value pairs.

5. We finally print the output result in the console by running the function grouped_dict_final_output(). This function calls the function print_element_console() for each element of `grouped_dict_final_output` to print out in the console:
- First, the key `'total'` key and its room repartition
- Second, the room names and their key repartition in alphabetical order as required in the `task_en.txt` file.


This wraps up the detailed description of the script code. Check out the next notebook for a complexity evaluation of the algorithm.